In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import animation
import networkx as nx
import numpy as np
import pandas as pd
from segmentflow import segment, view
from skimage import graph, measure
import trimesh
%matplotlib qt
%load_ext autoreload
%autoreload 2

In [2]:
# https://networkx.org/documentation/latest/auto_examples/3d_drawing/plot_3d_rotation_animation.html#sphx-glr-auto-examples-3d-drawing-plot-3d-rotation-animation-py
G = nx.dodecahedral_graph()
pos = nx.spectral_layout(G, dim=3)
nodes = np.array([pos[v] for v in G])
edges = np.array([(pos[u], pos[v]) for u, v in G.edges()])
def init():
    ax.scatter(*nodes.T, alpha=0.2, s=100, color="blue")
    for vizedge in edges:
        ax.plot(*vizedge.T, color="gray")
    ax.grid(False)
    ax.set_axis_off()
    plt.tight_layout()
    return

def _frame_update(index):
    ax.view_init(index * 0.2, index * 0.5)
    return

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

ani = animation.FuncAnimation(
    fig,
    _frame_update,
    init_func=init,
    interval=50,
    cache_frame_data=False,
    frames=100,
)
plt.show()

In [2]:
G = nx.dodecahedral_graph()
print(f'{type(G)=}')
pos = nx.spectral_layout(G, dim=3)
nodes = np.array([pos[v] for v in G])
edges = np.array([(pos[u], pos[v]) for u, v in G.edges()])
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.scatter(*nodes.T, alpha=0.2, s=100, color="blue")
for vizedge in edges:
    ax.plot(*vizedge.T, color="gray")
ax.grid(False)
ax.set_axis_off()
plt.tight_layout()
plt.show()

type(G)=<class 'networkx.classes.graph.Graph'>


In [3]:
img_dir_path = Path(
    f'../../PSAAP/alshibli_1551_study'
    f'/segmentflow_output/output_160_2mpd'
    f'/output_160_labeled_voxels')
print(img_dir_path.resolve())
imgs_labeled = segment.load_images(img_dir_path, file_suffix='tif')
rag = graph.RAG(label_image=imgs_labeled, connectivity=2)
print(f'{type(rag)=}')

C:\Users\gusb\Research\PSAAP\alshibli_1551_study\segmentflow_output\output_160_2mpd\output_160_labeled_voxels
Loading images...
--> Images loaded as 3D array:  (160, 240, 240)
type(rag)=<class 'skimage.graph._rag.RAG'>


In [11]:
# pos = nx.spectral_layout(rag, dim=3)
# nodes = np.array([pos[v] for v in rag])
# edges = np.array([(pos[u], pos[v]) for u, v in rag.edges()])
map_array = np.arange(imgs_labeled.max() + 1)
rag_labels = map_array[imgs_labeled]
regions = measure.regionprops(rag_labels)
for (n, data), region in zip(rag.nodes(data=True), regions):
    data['centroid'] = tuple(map(int, region['centroid']))
# Defining the end points of the edges
# The tuple[::-1] syntax reverses a tuple as matplotlib uses (x,y)
# convention while skimage uses (row, column)
edges = [[rag.nodes[n1]['centroid'][::-1], rag.nodes[n2]['centroid'][::-1]]
            for (n1, n2) in rag.edges()]


KeyError: 'centroid'

In [7]:
edges = rag.edges()
nodes = rag.nodes()
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.scatter(*np.array(nodes).T, alpha=0.2, s=100, color="blue")
for vizedge in edges:
    ax.plot(*np.array(vizedge).T, color="gray")
ax.grid(False)
ax.set_axis_off()
plt.tight_layout()
plt.show()


TypeError: scatter() got multiple values for argument 's'

In [ ]:
props_df = pd.DataFrame(columns=[
    'particleID',
    'n_voxels',
    'centroid',
    'min_slice',
    'max_slice',
    'min_row',
    'max_row',
    'min_col',
    'max_col'
])
regions = measure.regionprops(imgs_labeled)
# n_particles = len(regions)
for region in regions:
    # Get bounding slice, row, and column
    min_slice, min_row, min_col, max_slice, max_row, max_col = region.bbox
    # Get centroid coords in slice, row, col and reverse to get x, y, z
    centroid_x, centroid_y, centroid_z = (
        reversed([str(round(coord)) for coord in region.centroid])
    )
    props = {}
    props['particleID'] = region.label
    props['n_voxels']   = region.area
    props['centroid_x'] = centroid_x
    props['centroid_y'] = centroid_y
    props['centroid_z'] = centroid_z
    props['min_slice']  = min_slice
    props['max_slice']  = max_slice
    props['min_row']    = min_row
    props['max_row']    = max_row
    props['min_col']    = min_col
    props['max_col']    = max_col
    props_df = pd.concat(
        [props_df, pd.DataFrame.from_records([props])], ignore_index=True
    )


In [ ]:
props_df = pd.DataFrame(columns=[
    'particleID',
    'n_voxels',
    'centroid',
    'min_slice',
    'max_slice',
    'min_row',
    'max_row',
    'min_col',
    'max_col',
    'n_voxels_convex',
    'convex_to_concave',
])
regions = measure.regionprops(imgs_labeled)
# n_particles = len(regions)
for region in regions:
    # Get bounding slice, row, and column
    min_slice, min_row, min_col, max_slice, max_row, max_col = region.bbox
    # Get centroid coords in slice, row, col and reverse to get x, y, z
    centroid_x, centroid_y, centroid_z = (
        reversed([str(round(coord)) for coord in region.centroid])
    )
    props = {}
    props['particleID'] = region.label
    props['n_voxels']   = region.area
    props['centroid_x'] = centroid_x
    props['centroid_y'] = centroid_y
    props['centroid_z'] = centroid_z
    props['min_slice']  = min_slice
    props['max_slice']  = max_slice
    props['min_row']    = min_row
    props['max_row']    = max_row
    props['min_col']    = min_col
    props['max_col']    = max_col
    props['n_voxels_convex'] = region.area_convex
    props['concavity'] = props['n_voxels_convex'] / props['n_voxels']
    props_df = pd.concat(
        [props_df, pd.DataFrame.from_records([props])], ignore_index=True
    )


In [ ]:
props_df = props_df.sort_values('concavity', ascending=False)
props_df

In [ ]:
props_df['concavity'].hist()

In [ ]:
i = props_df.iloc[0]['particleID']
print(props_df.iloc[i]['particleID'])
stl_path = Path(
    f'../../PSAAP/alshibli_1551_study'
    f'/segmentflow_output/output_160_2mpd'
    f'/output_160_STLs/output_160_{i:04}.stl')
print(stl_path)
mesh = trimesh.load(str(stl_path))
# mesh.visual.face_colors = [253,231,36,255]
# Establish scene
scene = trimesh.Scene()
scene.add_geometry(mesh)
scene.show()

In [ ]:
concave_idcs = props_df['particleID'][:50].to_list()
concave_fns = [f'output_160_{i}' for i in concave_idcs]
print(concave_fns)

In [ ]:
conditions = [
    # c <= 1
    (props_df['concavity'] <= 1),
    # 1 < c <= 1.1
    (props_df['concavity'] > 1) & (props_df['concavity'] <= 1.1),
    # 1.1 < c <= 1.2
    (props_df['concavity'] > 1.1) & (props_df['concavity'] <= 1.2),
    # 1.2 < c <= 1.3
    (props_df['concavity'] > 1.2) & (props_df['concavity'] <= 1.3),
    # 1.3 < c
    (props_df['concavity'] > 1.3),
]
for i, condition in enumerate(conditions):
    concave_idcs = props_df.loc[condition, 'particleID'].to_list()
    concave_fns = [f'output_160_{i:04d}' for i in concave_idcs]
    print(f'# N files: {len(concave_fns)}')
    print(f"bin_files['Bin {i}'] =", concave_fns)